# DOTr Waze Analysis (November 2018)
Code and supplementary documentation used to generate analysis.

In [1]:
import pandas as pd
import calendar
import datetime
import os

# Creating a local mmda_feed.csv file for analysis
I create another version using the master MMDA database as a source. I drop the `ds` column with `df.dropna` then I save it to CSV format.

In [ ]:
df = pd.read_csv(r'C:\Users\Panji\Documents\Python Scripts\Non-Jupyter Py Scripts\MMDA Tweet2Map\data_mmda_traffic_alerts.csv')
df['ds'] = df['Date'] + ' ' + df['Time']
df['ds'] = pd.to_datetime(df['ds'])
df = df[['ds', 'Location', 'Latitude', 'Longitude', 'Direction', 'Type', 'Lanes Blocked', 'Involved', 'Tweet', 'Source']]
df.dropna(subset=['ds'], inplace=True)
df.to_csv(r'C:\Users\Panji\Documents\Python Scripts\Non-Jupyter Py Scripts\DOTr\mmda_feed.csv')

# Step 1 - Create bins to separate data
### Create function to analyze the CSV files
Generate a CSV file for each hour of each day. The function `split_by_date` will go through the entire raw Waze CSV file. The size of the hour bin is dependent on the `bin_hour_size` argument. The timeframe is seen below as the `date_start` and `date_end` variable. These variables are what the while loop will depend on and the while loop will stop once it reaches the date stated in the `date_end` variable.

Each loop will save an output in a .CSV file whose filename will be determined by the `dfFilename` variable. This formula is:

` (Year) + (Month) + (Day) + ' ' + ('Time Lower Bound) + ('Time Upper Bound') + (File Extension .csv)`            

Where time bounds are in 24 hour format. For example, 20180401_0001 will read as 4 April 2018 00:00 - 01:00 AM.

### Functions for 24 hour bins vs non-24 hour bins
The `split_by_date` function does not function properly when a bin size of 24 is inputted. Thus a separate function based on the original code was made, `split_by_date_24hourbin`.

At the moment, the database that is labelled as `MMDA` serves as the test database while the `WAZE` database serves as the database that the test data will be tested against.

In [2]:
def split_by_date(df,database, date_start, date_end, bin_hour_size=1, print_statements=False):
    """
    df: Insert a Pandas dataframe
    database: 'MMDA' or 'WAZE'
    date format for date inputs: mm.dd.yyyy
    bin_hour_size: Size (in hours) of the bin
    """
    import os
    import shutil
    import time
    
    # Check database format
    if database == 'MMDA':
        databaseOutput = 'mmda_hourly'
    elif database == 'WAZE':
        databaseOutput = 'waze_hourly'
    else:
        raise ValueError(f'Invalid database argument: {database}')
    
    # Clean output folder
    folder = 'data\\' + databaseOutput
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)
    
    dateEnd = date_end
    dateStart = date_start
    # Get variables for the end dates
    dateEndMonth = int(dateEnd.split('.')[0])
    dateEndDay = int(dateEnd.split('.')[1])
    dateEndYear = int(dateEnd.split('.')[2])
    # Get variables for the start dates
    dateMonth = int(dateStart.split('.')[0])
    dateDay = int(dateStart.split('.')[1])
    dateYear = int(dateStart.split('.')[2])

    # Initial variables for loop
    runLoop = True
    timeLowerBound = 0
    timeUpperBound = 0 + bin_hour_size
    if print_statements == True:
        print('=================================')
        print(f'START {database} ANALYSIS')
        print('=================================')
    
    while runLoop == True:

        # print(f'{dateYear}{dateMonth}{dateDay}')
        # print(f'{timeLowerBound}{timeUpperBound}')
        if timeUpperBound < 25:
            # 24 hours in a day. If it detects the time variables reach 25, then it must go to the ELSE statement
            df_selection = df[((df['Timestamp'].dt.month == dateMonth) & (df['Timestamp'].dt.day == dateDay)) & 
               ((df['Timestamp'].dt.hour >= timeLowerBound) & (df['Timestamp'].dt.hour < timeUpperBound))]
        else:
            # Restart the time and move onto the next day
            if print_statements == True:
                print(f'Current Process: {dateMonth}.{dateDay}.{dateYear}')
            dateDay += 1
            timeLowerBound = 0
            timeUpperBound = timeLowerBound + bin_hour_size
            
        
        # Create filename for analysis in GIS software
        # Add 0 padding to day numbers with single digit
        if len(str(dateDay)) == 1:
            dateDayFile = '0' + str(dateDay)
        else:
            dateDayFile = dateDay
        if len(str(dateMonth)) == 1:
            dateMonthFile = '0' + str(dateMonth)
        else:
            dateMonthFile = dateMonth
        if len(str(timeLowerBound)) == 1:
            timeLowerBoundFile = '0' + str(timeLowerBound)
        else:
            timeLowerBoundFile = timeLowerBound
        if len(str(timeUpperBound)) == 1:
            timeUpperBoundFile = '0' + str(timeUpperBound)
        else:
            timeUpperBoundFile = timeUpperBound
        
        # Create appropriate filename, then save it
        dfFilename = (str(dateYear) + str(dateMonthFile) + str(dateDayFile) + '_' 
                      + str(timeLowerBoundFile) + str(timeUpperBoundFile) + '.csv')        
        df_selection.to_csv('data\\' + databaseOutput + '\\' + dfFilename, index=False)

        # Check if there are still days in the month. If not, proceed to next month
        if int(dateDay) > calendar.monthrange(dateYear, dateMonth)[1]:
            dateDay = 1
            dateMonth += 1
            if print_statements == True:
                print('--- NEW MONTH START')
            # print(f'Current Process: {dateMonth}.{dateDay}.{dateYear}')
        
        timeLowerBound += 1
        timeUpperBound += 1

        # Check to see if we've reached the end of the database
        if ((dateMonth == dateEndMonth) & (dateDay == dateEndDay) & (dateYear == dateEndYear)):
            break
    
    return print(f'{database} analysis done!\n')

### Code for the 24 hour bin

In [3]:
def split_by_date_24hourbin(df,database, date_start, date_end, print_statements=False):
    """
    Function for 24 hour bin analysis of traffic incident data
    df: Insert a Pandas dataframe
    database: 'MMDA' or 'WAZE'
    date format for date inputs: mm.dd.yyyy
    """
    import os
    import shutil

    # Check database format
    if database == 'MMDA':
        databaseOutput = 'mmda_hourly'
    elif database == 'WAZE':
        databaseOutput = 'waze_hourly'
    else:
        raise ValueError(f'Invalid database argument: {database}')
    
    # Clean output folder
    folder = 'data\\' + databaseOutput
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)
    
    dateEnd = date_end
    dateStart = date_start
    # Get variables for the end dates
    dateEndMonth = int(dateEnd.split('.')[0])
    dateEndDay = int(dateEnd.split('.')[1])
    dateEndYear = int(dateEnd.split('.')[2])
    # Get variables for the start dates
    dateMonth = int(dateStart.split('.')[0])
    dateDay = int(dateStart.split('.')[1])
    dateYear = int(dateStart.split('.')[2])

    # Variables for loop
    runLoop = True
    if print_statements == True:
        print('=================================')
        print(f'START {database} ANALYSIS')
        print('=================================')
    
    while runLoop == True:
        
        df_selection = df[((df['Timestamp'].dt.month == dateMonth) & (df['Timestamp'].dt.day == dateDay))]
        if print_statements == True:
            print(f'Current Process: {dateMonth}.{dateDay}.{dateYear}')
        
        # Create filename for analysis in GIS software
        # Add 0 padding to day numbers with single digit
        if len(str(dateDay)) == 1:
            dateDayFile = '0' + str(dateDay)
        else:
            dateDayFile = str(dateDay)
        if len(str(dateMonth)) == 1:
            dateMonthFile = '0' + str(dateMonth)
        else:
            dateMonthFile = dateMonth
            
        # Create appropriate filename, then save it
        dfFilename = (str(dateYear) + str(dateMonth) + str(dateDayFile) + '_' 
                      + str(0) + str(24) + '.csv')
        df_selection.to_csv('data\\' + databaseOutput + '\\' + dfFilename, index=False)

        # Check if there are still days in the month. If not, proceed to next month
        if int(dateDay) > calendar.monthrange(dateYear, dateMonth)[1]:
            dateDay = 1
            dateMonth += 1
            if print_statements == True:
                print('--- NEW MONTH START')
        else:            
            dateDay += 1

        if ((dateMonth == dateEndMonth) & (dateDay == dateEndDay) & (dateYear == dateEndYear)):
            break
    

    return print(f'{database} analysis done!\n')

# Apply function on Waze and MMDA Databases
2 separate cells. Cell directly below is only used only when using a 24 hour bin

In [4]:
### 24 HOUR BINS

## DRIVER
# df = pd.read_csv('DRIVER Accident Details (2).csv')
# df['Timestamp'] = pd.to_datetime(df['Date'])
# split_by_date_24hourbin(df=df, database='MMDA', date_start='10.31.2018', date_end='12.1.2018')

df = pd.read_csv('mmda_data_edited.csv')
df['Timestamp'] = df['Date'] + ' ' + df['Time']
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
split_by_date_24hourbin(df=df, database='MMDA', date_start='10.31.2018', date_end='12.1.2018')

df = pd.read_csv('waze_feed.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
split_by_date_24hourbin(df=df, database='WAZE', date_start='10.31.2018', date_end='12.1.2018')

MMDA analysis done!

WAZE analysis done!



Non-24 hour bin below

In [4]:
# Tweet2Map Database
df = pd.read_csv('mmda_data_edited.csv')
df['Timestamp'] = df['Date'] + ' ' + df['Time']
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
split_by_date(df=df, database='MMDA', date_start='10.31.2018', date_end='12.1.2018', bin_hour_size=3)

# # DRIVER Database
# df = pd.read_csv('DRIVER Accident Details (2).csv')
# df['Timestamp'] = pd.to_datetime(df['Date'])
# split_by_date(df=df, database='MMDA', date_start='10.31.2018', date_end='12.1.2018', bin_hour_size=3)

# Waze database
df = pd.read_csv('waze_feed.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
split_by_date(df=df, database='WAZE', date_start='10.31.2018', date_end='12.1.2018', bin_hour_size=3)

MMDA analysis done!

WAZE analysis done!



# Step 2: Creating a table to compare the databases
Then this process will merge the 2 datasets by creating 2 columns to link the datasets. The initial for loops will read all the file paths of all the files in the directory, store the unique file paths in a dictionary, then save the dictionary items to a .txt file.

Then it will generate a new .CSV file with 2 columns. One column of MMDA file paths and one column of Waze file paths. Each row will contain the file path of the waze/mmmda data and it will reference the same hour of the same day. This will prepare it for analysis using ArcGIS.

In [5]:
dataMMDA = {}
databaseMMDAFile = r'C:\Users\Panji\Documents\Python Scripts\Non-Jupyter Py Scripts\DOTr\data\mmda_hourly\00_dictionary_mmda.txt'
databaseMMDAPath = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\data\\mmda_hourly\\'
databaseMMDA = open(databaseMMDAFile, 'w')
dataWaze = {}
databaseWazeFile = r'C:\Users\Panji\Documents\Python Scripts\Non-Jupyter Py Scripts\DOTr\data\waze_hourly\00_dictionary_waze.txt'
databaseWazePath = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\data\\waze_hourly\\'
databaseWaze = open(databaseWazeFile, 'w')
dataCombined = []

# Waze Parser
# Read Waze CSV files and save the file path to a dictionary


for idx, file in enumerate(os.listdir(databaseWazePath)):
    if '.txt' not in file:
        dataWaze[idx] = file

for x, y in dataWaze.items():
    databaseWaze.writelines(str(x) + '/' + databaseWazePath + y + '\n')
databaseWaze.close()

# MMDA Parser
# Read MMDA CSV files and save the file path to a dictionary
for idx, file in enumerate(os.listdir(databaseMMDAPath)):
    if '.txt' not in file:
        dataMMDA[idx] = file

for x, y in dataMMDA.items():
    databaseMMDA.writelines(str(x) + '/' + databaseMMDAPath + y + '\n')
databaseMMDA.close()

# Merger process below
# list organization of dataJoin:
# item 0, Waze path
# item 1, MMDA path
for idxWaze, fileWaze in dataWaze.items():

    # print('idxWaze: {}'.format(idxWaze))

    with open(databaseMMDAFile) as f:
        contentMMDA = f.readlines()

    for x in contentMMDA:
        if str(idxWaze) == str(x.split('/')[0]):
            fileMMDA = x.split('/')[1]
            fileMMDA = fileMMDA.replace('\n', '')
            dataJoin = [databaseWazePath + fileWaze, fileMMDA]
            dataCombined.append(dataJoin)
            break

print('Database size: {}'.format(len(dataCombined)))

for x in dataCombined:
    wazeData = x[0]
    MMDAData = x[1]

df = pd.DataFrame(dataCombined,columns=['Waze','MMDA'])
df.to_csv('combined_table.csv', index=False)

Database size: 683


# Step 3: ArcGIS Buffer Analysis
**NOTE: YOU NEED THE ARCGIS ARCPY MODULE WHICH ONLY COMES WITH ARCGIS SOFTWARE**

This step will utilize the ArcPy package from ArcGIS to script and automate this buffer analysis.

The for loop will read the CSV file row by row. In each row it will extract the MMDA and Waze data and create a buffer which is specified in the `bufferDistance` string. During each loop, matching data will be appended written to a separate pandas dataframe which is stored in `df_selection` and then appended to `df_match` which is the variable that keeps track of total matches. The dataframe in `df_match` is then saved to a .csv file. This part is for creating a histogram of matching data.

An intersect analysis based and count the amount of MMDA point data that intersect with the Waze buffer. The output is not saved to any file but it is displayed as `print` statements and can be copied and pasted.

In [ ]:
# -*- coding: utf-8 -*-
import arcpy
import pandas as pd
import os
import time

# SCRIPT PARAMETERS
bufferDistance = "300 Meters"
runMatchRate = False
runIntersectionAnalysis = True

# Declare initial variables
scratch = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\scratch'
scratchExcel = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\scratch\\scratchCsv.xls'
df = pd.read_csv('combined_table.csv')
spRef = arcpy.SpatialReference("C:\\GIS\\Data Files\\Work Files\\MMDA Tweet2Map\\input\\GCS_WGS_1984.prj")
outputWaze = 'outputWaze'
outputMMDA = 'outputMMDA'
BufferWaze_shp = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\scratch\\BufferWaze.shp'
BufferMMDA_shp = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\scratch\\BufferMMDA.shp'
Buffer_Intersect_shp = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\scratch\\intersection.shp'
dataJoin = 'C:\\GIS\\Data Files\\Work Files\\scratch\\dotr_waze.gdb\\dataJoin'
dataMatches = 'C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\data\\matches'
wazeTotal = 0
MMDATotal = 0
MMDADuplicateTotal = 0
duplicateTotal = 0
firstLoop = True

# Process: Set workspace environment
print('Set workspace environment')
arcpy.env.workspace = r'C:\Users\Panji\Documents\Python Scripts\Non-Jupyter Py Scripts\DOTr\scratch'
print('Set output coordinate system')
arcpy.env.outputCoordinateSystem = spRef
arcpy.env.overwriteOutput = True

for idxMMDA, csvMMDA in enumerate(df['MMDA']):

    print('===============================================')

    # Get file from both MMDA and Waze
    for idxWaze, csvWaze in enumerate(df['Waze']):

        if (idxMMDA == idxWaze) and '.txt' not in csvWaze:
            fileMMDA = csvMMDA
            fileWaze = csvWaze
            break
        
    print('Waze: ' + fileWaze)
    print('MMDA: ' + fileMMDA)
    
    # print('Table to Table')
    arcpy.TableToTable_conversion(fileWaze, scratch, "tableWaze.dbf")
    arcpy.TableToTable_conversion(fileMMDA, scratch, "tableMMDA.dbf")

    # print('Make XY Event Layer')
    try:
        arcpy.MakeXYEventLayer_management(scratch + '\\tableWaze.dbf', "longitude", "latitude", outputWaze, spRef)
        arcpy.MakeXYEventLayer_management(scratch + '\\tableMMDA.dbf', "longitude", "latitude", outputMMDA, spRef)
    except:
        print('No data detected. Skipping set')
        continue
    
    # print('Buffer Analysis')
    arcpy.Buffer_analysis(outputWaze, BufferWaze_shp, bufferDistance, "FULL", "ROUND", "ALL", "", "PLANAR")
    arcpy.Buffer_analysis(outputMMDA, BufferMMDA_shp, bufferDistance, "FULL", "ROUND", "ALL", "", "PLANAR")

    # print('Intersect Analysis')
    arcpy.Intersect_analysis([BufferWaze_shp, outputMMDA], Buffer_Intersect_shp, "ALL", "", "POINT")
    
    if runMatchRate == True:
        # Keep track of the matches in a pandas dataframe
        # Get the initial dataframe running
        print('runMatchRate: {}'.format(runMatchRate))
        arcpy.TableToExcel_conversion(Buffer_Intersect_shp, scratchExcel)
        
        if firstLoop == True:
            df_match = pd.read_excel(scratchExcel)
            df_match = df_match[0:0]
            firstLoop = False
        
        df_selection = pd.read_excel(scratchExcel)
        df_match = df_match.append(df_selection)
        
    if runIntersectionAnalysis == True:
        # Keep track of the Waze data that matches to MMDA data
        # Waze point data inside MMDA buffer
        # Extract time and date to create a unique Excel file
        print('runIntersectionAnalysis: {}'.format(runIntersectionAnalysis))
        arcpy.SpatialJoin_analysis(BufferMMDA, outputWaze, wazeMatches, 'JOIN_ONE_TO_MANY', match_option='INTERSECT')
        fileWazeMatches = fileWaze.split('\\')[-1]
        fileWazeMatches = fileWazeMatches.split('.')[0]
        arcpy.TableToExcel_conversion(wazeMatches, dataMatches + fileWazeMatches + '.xls')

    print('Output Statistics')
    statIntersect = arcpy.GetCount_management(Buffer_Intersect_shp)
    statIntersect = int(statIntersect.getOutput(0))
    statWaze = arcpy.GetCount_management(outputWaze)
    statWaze = int(statWaze.getOutput(0))
    statMMDA = arcpy.GetCount_management(outputMMDA)
    statMMDA = int(statMMDA.getOutput(0))

     # Clean outputs
    print('Reset variables')
    arcpy.Delete_management("in_memory")
    arcpy.Delete_management('C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\scratch\\tableWaze.dbf')
    arcpy.Delete_management('C:\\Users\\Panji\\Documents\\Python Scripts\\Non-Jupyter Py Scripts\\DOTr\\scratch\\tableMMDA.dbf')

    # Statistics
    wazeTotal += statWaze
    MMDATotal += statMMDA
    duplicateTotal += statIntersect
    duplicateRatio = float(float(duplicateTotal) / float(MMDATotal))
    
    print('\n\nCURRENT SET:')
    print('Waze Incidents: {}'.format(statWaze))
    print('MMDA Incidents: {}'.format(statMMDA))
    print('Total duplicates: {}\n\n'.format(statIntersect))
    print('TOTAL SET:')
    print('Total Waze Incidents: {}'.format(wazeTotal))
    print('Total MMDA Incidents: {}'.format(MMDATotal))
    print('Total MMDA Duplicates: {}'.format(duplicateTotal))
    print('Duplicate/MMDA Total: {}/{}\n'.format(duplicateTotal, MMDATotal))
    print('Current MMDA Duplicate Ratio: {:.2f}'.format(duplicateRatio))
    print('Current matching dataframe shape: {}'.format(df_match.shape))

df_match.to_csv(r'C:\Users\Panji\Documents\Python Scripts\Non-Jupyter Py Scripts\DOTr\data\histograms\histogram.csv',
                index=False, encoding='utf-8')
print('===============================')
print('\nAnalysis done!')
print('Duplicate/MMDA Total: {}/{}'.format(duplicateTotal, MMDATotal))
print('Current MMDA Duplicate Ratio: {:.2f}\n'.format(duplicateRatio))
